In [2]:
import torch
print(torch.cuda.is_available())
print(torch.cuda.get_device_name(0) if torch.cuda.is_available() else "No GPU")


True
Tesla T4


In [3]:
!pip install transformers datasets sentencepiece evaluate rouge_score --quiet


  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 4.6 MB/s eta 0:00:00


In [4]:
from datasets import load_dataset

dataset = load_dataset("cnn_dailymail", "3.0.0")
dataset


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

3.0.0/train-00000-of-00003.parquet:   0%|          | 0.00/257M [00:00<?, ?B/s]

3.0.0/train-00001-of-00003.parquet:   0%|          | 0.00/257M [00:00<?, ?B/s]

3.0.0/train-00002-of-00003.parquet:   0%|          | 0.00/259M [00:00<?, ?B/s]

...

3.0.0/test-00000-of-00001.parquet:   0%|          | 0.00/30.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/287113 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/13368 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/11490 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['article', 'highlights', 'id'],
        num_rows: 287113
    })
    validation: Dataset({
        features: ['article', 'highlights', 'id'],
        num_rows: 13368
    })
    test: Dataset({
        features: ['article', 'highlights', 'id'],
        num_rows: 11490
    })
})

In [5]:
from transformers import T5Tokenizer
tokenizer = T5Tokenizer.from_pretrained("t5-small")
def preprocess_function(examples):
    inputs = ["summarize: " + doc for doc in examples["article"]]
    model_inputs = tokenizer(inputs, max_length=512, truncation=True)
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(examples["highlights"], max_length=128, truncation=True)
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs
tokenized_datasets = dataset.map(preprocess_function, batched=True)


tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


Map:   0%|          | 0/287113 [00:00<?, ? examples/s]

/usr/local/lib/python3.12/dist-packages/transformers/tokenization_utils_base.py:4034: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/13368 [00:00<?, ? examples/s]

Map:   0%|          | 0/11490 [00:00<?, ? examples/s]

In [7]:
import transformers
transformers.__version__


'4.57.1'

In [8]:
!pip install -U transformers

In [11]:
import os
os.environ["WANDB_DISABLED"] = "true"


In [15]:
from transformers import T5ForConditionalGeneration, TrainingArguments, Trainer
import torch

model = T5ForConditionalGeneration.from_pretrained("t5-small")
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)
training_args = TrainingArguments(
    output_dir="/content/t5_summarization",
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    num_train_epochs=1,
    save_strategy="epoch",
    eval_strategy="epoch",
    logging_dir="/content/logs",
    logging_strategy="steps",
    report_to="none",  )

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"].select(range(20000)),
    eval_dataset=tokenized_datasets["validation"].select(range(2000)),
    tokenizer=tokenizer,
    data_collator=data_collator,
)


trainer.train()


/tmp/ipython-input-3378174849.py:19: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss
1,2.055800,2.036332


TrainOutput(global_step=10000, training_loss=2.0767898681640626, metrics={'train_runtime': 985.6677, 'train_samples_per_second': 20.291, 'train_steps_per_second': 10.145, 'total_flos': 2672934494208000.0, 'train_loss': 2.0767898681640626, 'epoch': 1.0})

In [19]:
from evaluate import load
import numpy as np

rouge = load("rouge")

def evaluate_summaries(dataset, model, tokenizer, num_samples=100):
    samples = dataset.select(range(num_samples))
    inputs = [ex["article"] for ex in samples]
    refs = [ex["highlights"] for ex in samples]

    inputs_tokenized = tokenizer(
        inputs, return_tensors="pt", padding=True, truncation=True, max_length=512
    )
    outputs = model.generate(
        input_ids=inputs_tokenized["input_ids"].to(model.device),
        attention_mask=inputs_tokenized["attention_mask"].to(model.device),
        max_length=150,
        num_beams=4,
    )

    preds = tokenizer.batch_decode(outputs, skip_special_tokens=True)
    result = rouge.compute(predictions=preds, references=refs)
    result = {key: round(value * 100, 2) for key, value in result.items()}
    return result, preds, refs
results, preds, refs = evaluate_summaries(dataset["validation"], model, tokenizer)
print("ROUGE results:", results)


ROUGE results: {'rouge1': np.float64(27.82), 'rouge2': np.float64(11.22), 'rougeL': np.float64(19.81), 'rougeLsum': np.float64(23.75)}


In [20]:
for i in range(3):
    article = dataset["validation"][i]["article"]
    reference = dataset["validation"][i]["highlights"]

    inputs = tokenizer(article, return_tensors="pt", truncation=True, padding=True, max_length=512)
    summary_ids = model.generate(
        inputs["input_ids"].to(model.device),
        max_length=150,
        num_beams=4,
        early_stopping=True
    )
    generated_summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)

    print(f"\nOriginal Article:\n{article[:500]}...")
    print(f"\nReference Summary:\n{reference}")
    print(f"\nGenerated Summary:\n{generated_summary}")
    print("="*80)



Original Article:
(CNN)Share, and your gift will be multiplied. That may sound like an esoteric adage, but when Zully Broussard selflessly decided to give one of her kidneys to a stranger, her generosity paired up with big data. It resulted in six patients receiving transplants. That surprised and wowed her. "I thought I was going to help this one person who I don't know, but the fact that so many people can have a life extension, that's pretty big," Broussard told CNN affiliate KGO. She may feel guided in her ge...

Reference Summary:
Zully Broussard decided to give a kidney to a stranger .
A new computer program helped her donation spur transplants for six kidney patients .

Generated Summary:
"I thought I was going to help this one person who I don't know, but the fact that so many people can have a life extension, that's pretty big," Zully Broussard told CNN affiliate KGO. Five surgeons, a covey of physician assistants, nurses and anesthesiologists, and more than 40 support staff 